### 1. Scraping Cricket Teams Data

In [58]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

data = []

soup = BeautifulSoup(requests.get('https://www.cricbuzz.com/cricket-team').content,'html.parser')

# Data of all the  Teams
sp = soup.find('div', class_ = 'cb-col cb-col-100')                                       

# Data of Each Team displayed at Left
for team_data in sp.find_all('div', class_ = 'cb-col cb-col-50 cb-team-item cb-lst-itm cb-team-lft-item'):   
    team_name = team_data.text.strip()
    team_link = 'https://www.cricbuzz.com' + team_data.find('a').get('href') + '/players'
    data.append([team_name, team_link])

# Data of Each Team displayed at Right
for team_data in sp.find_all('div', class_ = 'cb-col cb-col-50 cb-team-item cb-lst-itm cb-team-rght-item'):   
    team_name = team_data.text.strip()
    team_link = 'https://www.cricbuzz.com' + team_data.find('a').get('href') + '/players'
    data.append([team_name, team_link])
    
df = pd.DataFrame(data, columns = ['name','link'])
df.to_csv('teams.csv',index = False)
print('teams.csv is created!')

teams.csv is created!


### 2. Scraping Players Data

In [63]:
players_data = []

for team_name, team_link in tqdm(data):           # Going through each team
    soup = BeautifulSoup(requests.get(team_link).content, 'html.parser')
    
    for player in soup.find('div', class_ = 'cb-col-67 cb-col cb-left cb-top-zero').find_all('a'): # Players of Team
        player_name = player.text.strip()
        player_link = 'https://www.cricbuzz.com' + player.get('href').strip()
        players_data.append([player_name, team_name, player_link])
        
        
df = pd.DataFrame(players_data, columns = ['player','team','link'])
df.to_csv('players.csv', index = False)
print('players.csv is created!')

100%|███████████████████████████████████████████| 37/37 [00:17<00:00,  2.08it/s]

players.csv is created!


### 3. Scraping  Players Data in Detail

In [115]:
final_data = []

for name, team, link in tqdm(players_data):

    details = []
    
    details.append(name)
    details.append(team)
    
    soup = BeautifulSoup(requests.get(link).content,'html.parser')
    
    try:
        # Basic Information
        basic_info = [sp.text.strip() for sp in soup.find('div', class_ = 'cb-col cb-col-33 text-black').find_all('div', class_ = 'cb-col cb-col-60 cb-lst-itm-sm')[:3]]
        details += basic_info


        # Data of Height
        height = soup.find('div', class_ = 'cb-col cb-col-33 text-black').find('div', class_ = 'cb-col cb-col-60').text.strip()
        details.append(height)

        # Ranks
        ranks = [sp.text.strip() for sp in soup.find('div', class_ = 'cb-col cb-col-33 text-black').find_all('div', class_ = 'cb-col cb-col-25 cb-plyr-rank text-right')]
        details += ranks


        details.append(link)    
        final_data.append(details)
        
    except:
        details = []
    
df = pd.DataFrame(final_data, columns = ['name','team','date_of_birth','palce_of_birth','role','height',
                                         'test_bat_rank','odi_bat_rank','t20_bat_rank',
                                         'test_bow_rank','odi_bow_rank','t20_bow_rank',
                                         'link'])

df.to_csv('cricket.csv', index = False)

100%|█████████████████████████████████████████| 453/453 [05:50<00:00,  1.29it/s]
